# TP 3 SciQLOP - Virtual Products
The goal of this practical session is to create and manipulate SciQLOP virtual products
<div align="center">
<img src="https://github.com/SciQLop/SciQLop/raw/main/SciQLop/resources/icons/SciQLop.png"/>
</div>

# Mirror mode threshold virtual products

In this exercice, we will create a virtual product that computes the mirror mode instability threshold value\ 
that we have already encountered in the previous exercice.\
As a reminder, the formula is :


$$ C = \beta_\perp\left(\frac{T_\perp}{T_{\parallel}}-1\right)$$ 


where :

$$\beta_\perp = \frac{2\mu_0P_\perp}{B^2}$$

Now in a SciQLOP notebook, define a function computing this value, and define a new scalar virtual product under the path "mms/mirror".

Complete the following code to create the virtual product.


# 

In [ ]:
from SciQLop.user_api.virtual_products import create_virtual_product, VirtualProductType
from speasy import SpeasyVariable
from speasy.signal.resampling import interpolate
import speasy as spz
import scipy.constants as cst


def mirror_mode_threshold(start_time: float, stop_time: float) -> SpeasyVariable or None:
    ...
    return ...


mirror_mode_threshold_vp = create_virtual_product("/mms/mirror", mirror_mode_threshold, VirtualProductType.Scalar,
                                                  labels=["Mirror mode threshold"])

In [ ]:

# ANSWER
# ------

from SciQLop.user_api.virtual_products import create_virtual_product, VirtualProductType
from speasy import SpeasyVariable
from speasy.signal.resampling import interpolate
import speasy as spz
import scipy.constants as cst


def mirror_mode_threshold(start_time: float, stop_time: float) -> SpeasyVariable or None:
    mms1_products = spz.inventories.data_tree.cda.MMS.MMS1
    products = [mms1_products.DIS.MMS1_FPI_FAST_L2_DIS_MOMS.mms1_dis_temppara_fast,
                mms1_products.DIS.MMS1_FPI_FAST_L2_DIS_MOMS.mms1_dis_tempperp_fast,
                mms1_products.FGM.MMS1_FGM_SRVY_L2.mms1_fgm_b_gse_srvy_l2,
                mms1_products.DIS.MMS1_FPI_FAST_L2_DIS_MOMS.mms1_dis_numberdensity_fast]

    tpara, tperp, b, n = spz.get_data(products, start_time, stop_time)

    anisotropy = tperp / tpara
    Pperp = tperp * n * 1e6
    b = interpolate(tperp, b)
    betaperp = Pperp * cst.mu_0 * cst.e * 2 / (b["Bt"] * 1e-9) ** 2
    mirror = betaperp * (anisotropy - 1)
    return mirror


mirror_mode_threshold_vp = create_virtual_product("/mms/mirror", mirror_mode_threshold, VirtualProductType.Scalar,
                                                  labels=["Mirror mode threshold"])


In [ ]:
from SciQLop.user_api.plot import create_plot_panel
from SciQLop.user_api import TimeRange

panel = create_plot_panel()
panel.time_range = TimeRange("2021-01-01", "2021-01-02")
panel.plot(mirror_mode_threshold_vp)